<h1>Singularity Extractor</h1>
By <a href="https://movsisyan.info/">Mher Movsisyan</a>

## 1 - Importing Tools and Data

In [2]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from keras import backend as K
import matplotlib.pyplot as plt
from keras.models import Sequential
import matplotlib.patches as patches
from skimage.filters import threshold_otsu
from keras.optimizers import RMSprop, SGD, Adam
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras.layers.experimental.preprocessing import RandomTranslation, RandomRotation, RandomWidth, RandomHeight, RandomZoom, Resizing
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPool2D, BatchNormalization, Reshape, GlobalAveragePooling2D, LeakyReLU, Layer

seed = 173
np.random.seed(seed)
tf.random.set_seed(seed)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1131276721359329939
]


In [3]:
df = pd.read_csv("../data/drawings_non_binary.csv")
mnist = pd.read_csv("../data/train.csv")
df

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5505,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
5506,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
5507,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
5508,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9


## 2 - Joining Data

In [4]:
joint = pd.DataFrame(mnist.iloc[:, 1:].to_numpy())
joint["label"] = mnist.label
jointDf = pd.DataFrame(np.concatenate((joint, df), axis=0)).rename({784: "label"}, axis=1).sample(frac=1, random_state=seed)

One-hot encoding labels

In [5]:
y = to_categorical(jointDf.label)

Extracting inference and holdout datasets

In [6]:
x_train, x_test, y_train, y_test = train_test_split(jointDf.iloc[:,:-1], y, train_size=0.9, random_state=seed) # no need to stratify for large balanced datasets
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((42759, 784), (4751, 784), (42759, 10), (4751, 10))

In [7]:
class GateOfLearning(Callback):
    """Increases learning rate when stuck at extrema, a friend to ReduceLROnPlateau, ModelCheckpoint callbacks.
    
    
    Example:
        ```python
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                    patience=5, min_lr=0.001)
        increase_lr = GateOfLearning(monitor='val_loss', factor=25,
                                      patience=8, max_lr=0.05)
        model.fit(X_train, Y_train, callbacks=[increase_lr, reduce_lr])
        ```
    
    
    Args:
        monitor: quantity to be monitored.
        factor: factor by which the learning rate will be reduced. Must be multitudes greater than that of the ReduceLROnPlateau
            `new_lr = lr * factor`.
        patience: number of epochs with no improvement after which learning rate will be increased. Must be greater than that of the ReduceLROnPlateau (6 by default)
        verbose: int. 0: quiet, 1: update messages. (1 by default)
        mode: one of `{'min', 'max'}`. In `'min'` mode, the learning rate will be increased when the quantity monitored has stopped decreasing; in `'max'` mode it will be increased when the quantity monitored has stopped increasing.
        cooldown: number of epochs to wait before resuming normal operation afterlr has been reduced. (0 by default)
        max_lr: upper bound on the learning rate. (initial value * factor * 2 by default)
        sleep_till_reduce: do nothing untill seeing a reduction in the learning rate
    """
    
    
    def __init__(self, monitor = "val_loss", factor = 15.0, patience=6, verbose=1, mode = "min", cooldown = 0, max_lr = 999):
        # Sanity check
        if factor <= 1.0:
            raise ValueError("GateOfLearning does not support a factor <= 1.0.")
        
        if mode not in ["min", "max"]:
            raise ValueError(f"GateOfLearning does not support a mode '{mode}'. Use 'min' or 'max' instead.")
        
        # Init
        super(GateOfLearning, self).__init__()
        self.monitor = monitor
        self.factor = factor
        self.patience = patience
        self.verbose = verbose
        self.objective = min if mode == "min" else max
        self.cooldown = cooldown
        self.max_lr = max_lr
        
        self.backup = (monitor, factor, patience, verbose, mode, cooldown, max_lr)
        
        self.observations = []
        self.lr_history = []
        self.last_opened = 0
    
    
    def _reset(self):
        """Reset state"""
        self.monitor, self.factor, self.patience, self.verbose, self.mode, self.cooldown, self.max_lr = self.backup
        
        self.observations = []
        self.lr_history = []
        self.last_opened = 0
        
    
    def on_train_begin(self, logs=None):
        """Training start handler"""
        self._reset()
    
    
    def open_gate(self):
        """Increases learning rate"""
        new_lr = self.lr_history[-1] * self.factor
        
        assert new_lr > self.lr_history[-1], f"old: {self.lr_history[-1]}, new: {new_lr}"
        
        if new_lr > self.max_lr:
            
            if self.verbose:
                print("Learning rate diverged. You can solve this problem by using a faster ReduceLROnPlateau, a smaller factor, or a bigger patience/cooldown.")
        else:
            old_lr = float(self.model.optimizer.learning_rate)
            self.model.optimizer.learning_rate = new_lr
            if self.verbose:
                print(f"\nGateOfLearning: Learning rate increased from {old_lr} to {float(self.model.optimizer.learning_rate)}")
    
    def on_epoch_end(self, epoch, logs=None):
        """Epoch end handler"""
        # Log learning rate.
        self.lr_history.append(logs["lr"])
        
        # Set the maximum learning rate to the initial or otherwise specified maximum learning rate
        if len(self.lr_history) <= 1:
            self.max_lr = min(self.max_lr, self.factor * 2 * self.lr_history[0])
        
        # Check if the metric is reported, otherwise use default metrics.
        if self.monitor not in logs.keys():
            initMetric = self.monitor
            self.monitor = "val_loss" if "val_loss" in logs.keys() else "loss"
            if self.verbose:
                print(f"\nGateOfLearning: The '{initMetric}' metric was never reported. Using '{self.monitor}' instead.\n")
        
        # Log metric
        self.observations.append(logs[self.monitor])
        
        # Check if it is too early for an opening
        if len(self.observations) <= self.patience:
            return
        
        # Check if there is no improvement
        if self.objective(self.observations[-self.patience:]) == self.observations[-self.patience]:
            if epoch - self.last_opened > self.cooldown:
                self.open_gate()
                self.last_opened = epoch
                self.observations = [self.observations[-self.patience]]

In [8]:
class SingularityExtractor2D(Layer):
    """
    Counts small patches of dark pixels that are surrounded by light pixels
    """

    def __init__(self, degree=6, kernel_size=3, strides=1, padding="SYMMETRIC", margin=1, **kwargs):
        if not ((kernel_size % 2) and kernel_size >= 3 and isinstance(kernel_size, int)):
            raise ValueError("kernel_size: value must be odd, >= 3")

        if not (strides >= 1 and isinstance(strides, int)):
            raise ValueError("strides: must be integer >= 1")

        if not (margin >= 0 and isinstance(margin, int)):
            raise ValueError("margin: must be a non-negative integer")

        if padding not in ["CONSTANT", "SYMMETRIC", "REFLECT"]:
            raise ValueError(
                "padding: must be one of ['CONSTANT', 'SYMMETRIC', 'REFLECT']")

        self.degree = degree
        self.kernel_size = kernel_size
        self.strides = strides
        self.padding = padding
        self.margin = margin + 1

        self.radius = int((kernel_size - 1)/2) * strides

        super(SingularityExtractor2D, self).__init__(**kwargs)

    # region Keras api

    def build(self, input_shape):
        if len(input_shape) != 4:
            raise ValueError("input must be 4D (batch_size, x, y, 1)")

        super(SingularityExtractor2D, self).build(input_shape)

    def call(self, input_data):
        return self.extract(input_data, "keras")

    def compute_output_shape(self, input_shape):
        return (input_shape[-3], input_shape[-2] - 2 * self.margin, input_shape[-1] - 2 * self.margin)

    # endregion Keras api

    # region Scikit api

    def fit(self, x, *args, **kwargs):
        if len(x.shape) != 3:
            raise ValueError("input must be 3D (number_of_observations, x, y)")

    def transform(self, input_data, *args, **kwargs):
        return self.extract(input_data, "sk")

    # endregion Scikit api

    def extract(self, input_data, api):
        outputs = []

        if api == "sk":
            pass
        else:
            matpad = tf.pad(input_data, [[0, 0], [self.radius, self.radius], [self.radius, self.radius], [0, 0]], mode=self.padding)
            print(matpad.shape[0])
            ones = tf.ones((matpad.shape[0] or 0, self.kernel_size, 1, 1))

            conv = tf.nn.convolution(matpad, ones, self.strides, padding="SAME", dilations=1)
            
            return ((conv[:, 1:-1, 1:-1, :] - input_data) / conv[:, 1:-1, 1:-1, :]) ** self.degree

In [9]:
def get_model(seed):
        model = Sequential([
                # Preprocessing
                Reshape((28, 28, 1), input_shape=(1, 28*28)),
                RandomRotation(0.08, seed=seed, fill_mode="constant", fill_value=0),
                RandomWidth(0.08, seed=seed, interpolation="bicubic"),
                RandomTranslation(0.05, 0.28, seed=seed, fill_mode="constant", fill_value=0),
                Resizing(28, 28, interpolation="bicubic"),
                
                SingularityExtractor2D(6, 3, 1, "SYMMETRIC", 1),
                # Decision-making
                Flatten(),
                Dense(128),
                LeakyReLU(0.1),
                BatchNormalization(axis=1),
                Dropout(0.4, seed=seed),
                Dense(10, activation="softmax")])
        
        opt = RMSprop(learning_rate = 0.002, decay = 0)
        model.compile(opt, "categorical_crossentropy", metrics=["accuracy"])

        return model

lr_cut = ReduceLROnPlateau(monitor="val_accuracy", factor=0.3, patience=2, verbose=1, min_lr=0.000000001, mode="max", cooldown=3)
quicksave = ModelCheckpoint(monitor="val_accuracy", filepath="data/checkpoint.hdf5", verbose=1, save_best_only=True)
lr_boost = GateOfLearning(monitor="val_accuracy", factor=30, patience = 8, mode = "max", cooldown=9)

model = get_model(seed)
model.summary()


None
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
random_rotation (RandomRotat (None, 28, 28, 1)         0         
_________________________________________________________________
random_width (RandomWidth)   (None, 28, None, 1)       0         
_________________________________________________________________
random_translation (RandomTr (None, 28, None, 1)       0         
_________________________________________________________________
resizing (Resizing)          (None, 28, 28, 1)         0         
_________________________________________________________________
singularity_extractor2d (Sin (None, 28, 28, 1)         0         
_________________________________________________________________
flatten (Flatten)            (None, 784)           

In [10]:
hist = model.fit(x_train[:4000]/255, y_train[:4000], epochs=7, batch_size=128, validation_data = (x_test[:400]/255, y_test[:400]), shuffle=True, callbacks=[lr_cut, quicksave, lr_boost])

Epoch 1/7
None
None
